## Project 4 - Treasure Hunters Inc.
###  We do the treasure hunting and monster fighting for you
1. Set up a new git repository in your GitHub account
2. Think up a map-like environment with treasure, obstacles and opponents
3. Choose a programming language (Python, C/C++, Java)
4. Formulate ideas on how reinforcement learning can be used to find treasure efficiently while avoiding obstacles and opponents
5. Build one or more reinforcement policies to model situational assessments, actions and rewards programmatically
6. Document your process and results
7. Commit your source code, documentation and other supporting files to the git repository in GitHub

# Step 1: Importing libraries
- Here we only need numpy and copy for copying a list

In [1]:
import numpy as np
import copy

# Step 2: Defining the Environment for our model

### Map :
- Map is of 7rows and 7 columns
- total of 49 possible states

### Q-Values:
- Initializing a 3 dimensional array with zeros to hold Q-values for each state and action pair Q(s,a)
- This array consists of 7 rows and 7 columns and in 3rd dimension we have all possible actions

In [2]:
environment_rows = 7
environment_columns = 7

q_values = np.zeros((environment_rows, environment_columns, 4))

## Actions:

#### We have four possible actions for the agent:
- up(value:0)
- right(value:1)
- down(value:2)
- left(value:3)

In [3]:
actions = ['up', 'right', 'down', 'left']

## Reward Table:

- Creating a two-dimensional reward table which can hold reward for every state
- This reward table's shape matches our map's shape


## Rewards

- Let us punish with -50, if agent encounters its opponent
- Let us punish with -30, if agent crashes into obstacle
- Let us reward with 0, if agent finds treasure
- Let us reward with 100, if agent reaches goal state
- To motivate agent to finish quick, we shall punish every step with -1

In [4]:
OPPONENT_REWARD = -50
TREASURE_REWARD = 0
OBSTACLE_REWARD = -10
EMPTY_REWARD = -1
GOAL_REWARD= 100

rewards = [
            [EMPTY_REWARD, EMPTY_REWARD, EMPTY_REWARD, EMPTY_REWARD, TREASURE_REWARD, OPPONENT_REWARD, OBSTACLE_REWARD],
            [TREASURE_REWARD, OPPONENT_REWARD, EMPTY_REWARD, EMPTY_REWARD, EMPTY_REWARD, EMPTY_REWARD, OPPONENT_REWARD],
            [EMPTY_REWARD, EMPTY_REWARD, TREASURE_REWARD, EMPTY_REWARD, OBSTACLE_REWARD, EMPTY_REWARD, EMPTY_REWARD],
            [EMPTY_REWARD, EMPTY_REWARD, OPPONENT_REWARD, EMPTY_REWARD, OBSTACLE_REWARD, EMPTY_REWARD, EMPTY_REWARD],
            [EMPTY_REWARD, TREASURE_REWARD, EMPTY_REWARD, EMPTY_REWARD, OBSTACLE_REWARD, EMPTY_REWARD, EMPTY_REWARD],
            [EMPTY_REWARD, EMPTY_REWARD, EMPTY_REWARD, TREASURE_REWARD, OBSTACLE_REWARD, EMPTY_REWARD, EMPTY_REWARD],
            [EMPTY_REWARD, EMPTY_REWARD, EMPTY_REWARD, EMPTY_REWARD, EMPTY_REWARD, EMPTY_REWARD, GOAL_REWARD],
        ]
for row in rewards:
    print(row)

## Visualizing Environment

In [5]:
OPPONENT = '_E_'
TREASURE = '_$_'
OBSTACLE = '|-|'
EMPTY = '---'
PLAYER = '_P_'
PLAYERGOLD = '$P$'
PLAYERGOAL = '^P^'
GOAL= 'END'
Visual = [
            [EMPTY, EMPTY, EMPTY, EMPTY, TREASURE, OPPONENT, OBSTACLE],
            [TREASURE, OPPONENT, EMPTY, EMPTY, EMPTY, EMPTY, OPPONENT],
            [EMPTY, EMPTY, TREASURE, EMPTY, OBSTACLE, EMPTY, EMPTY],
            [EMPTY, EMPTY, OPPONENT, EMPTY, OBSTACLE, EMPTY, EMPTY],
            [EMPTY, TREASURE, EMPTY, EMPTY, OBSTACLE, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY, TREASURE, OBSTACLE, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, EMPTY, GOAL],
        ]
print(f"Labels:::>\n\nOpponent: '{OPPONENT}'\nTreasure: '{TREASURE}'\nOBSTACLE: '{OBSTACLE}'\nEmpty space: '{EMPTY}'\n")
for row in Visual:
    print(row)

# Step-3: Helper Functions:

## To check if the current state is terminal

In [6]:
def isTerminalState(current_row, current_column):
    if rewards[current_row][current_column] == EMPTY_REWARD:
        return False  
    elif rewards[current_row][current_column] == TREASURE_REWARD:
        return False
    else:
        return True

## To get starting location
- Everytime we start our agent in the map from a random state which is not terminal
- this is to help our agent to learn how to reach goal state from any non-terminal state after training.
- loop until we find a non-terminal state

In [7]:
def getStartingLocation():
    current_row = np.random.randint(environment_rows)
    current_column = np.random.randint(environment_columns)
    while isTerminalState(current_row, current_column):
        current_row = np.random.randint(environment_rows)
        current_column = np.random.randint(environment_columns)
    return current_row, current_column

## Get next action to perform
- if a randomly chosen value between 0 and 1 is less than epsilon, then choose the most promising value from the Q-table for this state. This is `Exploitation`
- else we choose a random possible action. This is `Exploration`

In [8]:
def getNextAction(current_row, current_column, epsilon):
    if np.random.random() < epsilon:
        return np.argmax(q_values[current_row][current_column])
    else: #choose a random action
        return np.random.randint(4)

## Get next location
- based on the action we chose, this function will return the next state.

In [9]:
def getNextLocation(current_row, current_column, action_index):
    new_row = current_row
    new_column = current_column
    if actions[action_index] == 'up' and current_row > 0:
        new_row -= 1
    elif actions[action_index] == 'right' and current_column < environment_columns - 1:
        new_column += 1
    elif actions[action_index] == 'down' and current_row < environment_rows - 1:
        new_row += 1
    elif actions[action_index] == 'left' and current_column > 0:
        new_column -= 1
    return new_row, new_column

## Get optimal path:
- This function returns the optimal path to goal, from the non-terminal start state that we choose 

In [10]:
def getPath(start_row, start_column):
    if isTerminalState(start_row, start_column):
        return []
    else: 
        current_row, current_column = start_row, start_column
        shortest_path = []
        shortest_path.append([current_row, current_column])
        while not isTerminalState(current_row, current_column):
            action_index = getNextAction(current_row, current_column, 1.)
            current_row, current_column = getNextLocation(current_row, current_column, action_index)
            shortest_path.append([current_row, current_column])
        return shortest_path

# Step-4 : Training our agent
- we are training on 10,000 episodes to get more optimal results

In [11]:
#defining parameters
#the percentage of time when we should take the best action (instead of a random action)
epsilon = 0.9 
#discount factor for future rewards
discount_factor = 0.9 
#the rate at which the agent should learn
learning_rate = 0.9 


for episode in range(10000):
    row_index, column_index = getStartingLocation()

    #continue moving until we reach a terminal state
    while not isTerminalState(row_index, column_index):
        
        #choose which action to take
        action_index = getNextAction(row_index, column_index, epsilon)

        #transition to the next state and store the old row and column indexes
        old_row_index, old_column_index = row_index, column_index 
        row_index, column_index = getNextLocation(row_index, column_index, action_index)

        #receive the reward for moving to the new state
        reward = rewards[row_index][column_index]
        
        # calculate the temporal difference
        old_q_value = q_values[old_row_index, old_column_index, action_index]
        temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

        #update the Q-value in q-table
        new_q_value = old_q_value + (learning_rate * temporal_difference)
        q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Training completed')

Training completed


# Step5: Testing our model

In [12]:
print(getPath(2, 1)) #starting at row 9, column 5
print(getPath(0,3)) #starting at row 9, column 5

[[2, 1], [2, 2], [2, 3], [3, 3], [4, 3], [5, 3], [6, 3], [6, 4], [6, 5], [6, 6]]
[[0, 3], [0, 4], [1, 4], [1, 5], [2, 5], [2, 6], [3, 6], [4, 6], [5, 6], [6, 6]]


# Step6: Visualizing agent path

In [13]:
def visualizeRoute(path,track=Visual):
    
    print(f"::::Labels:::>\n\nOpponent: '{OPPONENT}'\nTreasure: '{TREASURE}'\nOBSTACLE: '{OBSTACLE}'\nEmpty   : '{EMPTY}'")
    print(f"Agent  : '{PLAYER}'")    
    print(f"Agent with treasure : '{PLAYERGOLD}'")
    print(f"Agent reached goal : '{PLAYERGOAL}'")
    print("\nInitial Map")
    for row in Visual:
        print(row)
    print(f'Result:\nstarting point: row->{path[0][0]+1} column->{path[0][1]+1}')
    tVisual=copy.deepcopy(Visual)
    goldCount=0
    for step in path:
        if tVisual[step[0]][step[1]]==EMPTY:
            tVisual[step[0]][step[1]]=PLAYER
        elif tVisual[step[0]][step[1]]==TREASURE:
            tVisual[step[0]][step[1]]=PLAYERGOLD
            goldCount+=1
        elif tVisual[step[0]][step[1]]==GOAL:
            tVisual[step[0]][step[1]]=PLAYERGOAL
        else:
            print('ERROR')
    for row in tVisual:
        print(row)
    print(f'total steps: {len(path)}')
    print(f'total treasure collected {goldCount}')
    

## What is the optimal path if started from 3rd row and 2nd column

In [14]:
visualizeRoute(getPath(2,1))

::::Labels:::>

Opponent: '_E_'
Treasure: '_$_'
OBSTACLE: '|-|'
Empty   : '---'
Agent  : '_P_'
Agent with treasure : '$P$'
Agent reached goal : '^P^'

Initial Map
['---', '---', '---', '---', '_$_', '_E_', '|-|']
['_$_', '_E_', '---', '---', '---', '---', '_E_']
['---', '---', '_$_', '---', '|-|', '---', '---']
['---', '---', '_E_', '---', '|-|', '---', '---']
['---', '_$_', '---', '---', '|-|', '---', '---']
['---', '---', '---', '_$_', '|-|', '---', '---']
['---', '---', '---', '---', '---', '---', 'END']
Result:
starting point: row->3 column->2
['---', '---', '---', '---', '_$_', '_E_', '|-|']
['_$_', '_E_', '---', '---', '---', '---', '_E_']
['---', '_P_', '$P$', '_P_', '|-|', '---', '---']
['---', '---', '_E_', '_P_', '|-|', '---', '---']
['---', '_$_', '---', '_P_', '|-|', '---', '---']
['---', '---', '---', '$P$', '|-|', '---', '---']
['---', '---', '---', '_P_', '_P_', '_P_', '^P^']
total steps: 10
total treasure collected 2


## What is the optimal path if started from 1st row and 4th column

In [15]:
visualizeRoute(getPath(0,3))

::::Labels:::>

Opponent: '_E_'
Treasure: '_$_'
OBSTACLE: '|-|'
Empty   : '---'
Agent  : '_P_'
Agent with treasure : '$P$'
Agent reached goal : '^P^'

Initial Map
['---', '---', '---', '---', '_$_', '_E_', '|-|']
['_$_', '_E_', '---', '---', '---', '---', '_E_']
['---', '---', '_$_', '---', '|-|', '---', '---']
['---', '---', '_E_', '---', '|-|', '---', '---']
['---', '_$_', '---', '---', '|-|', '---', '---']
['---', '---', '---', '_$_', '|-|', '---', '---']
['---', '---', '---', '---', '---', '---', 'END']
Result:
starting point: row->1 column->4
['---', '---', '---', '_P_', '$P$', '_E_', '|-|']
['_$_', '_E_', '---', '---', '_P_', '_P_', '_E_']
['---', '---', '_$_', '---', '|-|', '_P_', '_P_']
['---', '---', '_E_', '---', '|-|', '---', '_P_']
['---', '_$_', '---', '---', '|-|', '---', '_P_']
['---', '---', '---', '_$_', '|-|', '---', '_P_']
['---', '---', '---', '---', '---', '---', '^P^']
total steps: 10
total treasure collected 1


## What is the optimal path if started from 1st row and 1stcolumn

In [16]:
visualizeRoute(getPath(0,0))

::::Labels:::>

Opponent: '_E_'
Treasure: '_$_'
OBSTACLE: '|-|'
Empty   : '---'
Agent  : '_P_'
Agent with treasure : '$P$'
Agent reached goal : '^P^'

Initial Map
['---', '---', '---', '---', '_$_', '_E_', '|-|']
['_$_', '_E_', '---', '---', '---', '---', '_E_']
['---', '---', '_$_', '---', '|-|', '---', '---']
['---', '---', '_E_', '---', '|-|', '---', '---']
['---', '_$_', '---', '---', '|-|', '---', '---']
['---', '---', '---', '_$_', '|-|', '---', '---']
['---', '---', '---', '---', '---', '---', 'END']
Result:
starting point: row->1 column->1
['_P_', '---', '---', '---', '_$_', '_E_', '|-|']
['$P$', '_E_', '---', '---', '---', '---', '_E_']
['_P_', '_P_', '$P$', '_P_', '|-|', '---', '---']
['---', '---', '_E_', '_P_', '|-|', '---', '---']
['---', '_$_', '---', '_P_', '|-|', '---', '---']
['---', '---', '---', '$P$', '|-|', '---', '---']
['---', '---', '---', '_P_', '_P_', '_P_', '^P^']
total steps: 13
total treasure collected 3


# Conclusion and Future Developments:

- Our model performed best with the current reward policy we have good results in the end.
- In future we can try to move the enemy everytime and agent has to escape enemy.